## sprint11_CNN_2d

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
import math

pd.set_option('display.max_columns', 250)
np.set_printoptions(threshold=np.inf)

In [3]:
class Conv_Initializer:
    def WI(self, output_channels_num, input_channels_num, F):
        WI = np.random.randn(output_channels_num, input_channels_num, F[0],F[1])
        return WI
    def BI(self, output_channels_num):
        ### ２次元、３次元を１とする。Wとの演算でのシェイプを合わせるため### 
        BI = np.random.randn(output_channels_num,1,1) 
        return BI
    
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)
    def B(self, n_nodes2):
        return np.random.randn(n_nodes2, 1).T

In [4]:
class SGD2:
    def __init__(self, lr=0.02):
        self.lr = lr
    def update(self, layer):
        layer.W = layer.W - self.lr * layer.dW
        layer.B = layer.B - self.lr * layer.dB.mean(axis=0)

#        print(layer.W.shape,layer.B.shape)

## 【問題1】2次元畳み込み層の作成
## 【問題2】2次元畳み込み後の出力サイズ(get_output_shape関数)

In [58]:
#SimpleConv2d(Conv_Initializer(), SGD2(0.01), F=[3, 3], S=[1,1], input_channels_num=1, output_channels_num=3)

class SimpleConv2d:
    def __init__(self, initializer, optimizer, F, S, input_channels_num=1, output_channels_num=3):

        self.initializer = initializer
        self.optimizer = optimizer
        self.F = F
        self.W = self.initializer.WI(output_channels_num, input_channels_num, self.F) 
        self.B = self.initializer.BI(output_channels_num)
        self.dW = np.zeros((output_channels_num, input_channels_num, self.F[0],self.F[1]))
        self.dB = np.zeros((output_channels_num,1,1))
        
        self.P = [0,0]
        self.S = S
    
    def get_output_shape(self):
        self.output_h = int((self.Z.shape[1] + self.P[0]*2 - self.F[0]) / self.S[0] + 1)
        self.output_w = int((self.Z.shape[2] + self.P[1]*2 - self.F[1]) / self.S[1] + 1)
        return self.output_h, self.output_w
                
    def forward(self, Z):
        # Z = (入力チャネル数、1d入力、2d入力)
        # A = (出力チャネル数、1d出力、2d出力)
        self.Z = Z
        self.get_output_shape()
        self.A = np.zeros([self.W.shape[0], self.output_h, self.output_w])
        
        # 出力Ｃ、入力Ｃ，ｈ、ｗをforループで回す。
        output_c_conv = []
        for output_c in range(self.W.shape[0]):
            convs = np.zeros((self.W.shape[1], self.output_h, self.output_w))
            for input_c in range(self.W.shape[1]):
                for i in range(self.output_h):
                    for j in range(self.output_w):
                        convs[input_c][i][j] = np.sum(self.Z[input_c][i:i+self.F[0],j:j+self.F[1]] * self.W[output_c][input_c]) 
            output_c_conv.append(convs)            
        self.A = (np.sum(output_c_conv, axis=1) + self.B)
        return self.A

# convs[input_c][i][j] = np.sum(self.Z[input_c][i:i+self.F[0]][j:j+self.F[1]] * self.W[output_c][input_c]) と記載をするとエラー。
# 別のコードとしては、self.A[output_c][i][j] = np.sum(self.Z[input_c][i:self.F[0]+i,j:self.F[1]+j] * self.W[output_c][input_c])
# 配列をappendしてlist化したものを、np.sum(axis=1)で配列にしている。self.Bのshapeは（出力チャネル数、1、1）としなければならない。
    
    def backward(self, dA):
        self.dB = np.sum(np.sum(dA,axis=1),axis=1)
        self.dZ = np.zeros((self.Z.shape[0], self.Z.shape[1]+2, self.Z.shape[2]+2))
        dA_pad = np.array([np.pad(x,1,"constant") for x in dA])

        # for文 input_c:元の入力、output_c:元の出力、i: Wの出力チャネルサイズ数
        for input_c in range(self.W.shape[1]):
            for output_c in range(self.W.shape[0]):
                for i in range(self.output_h):                
                    for j in range(self.output_w):
                        self.dW[output_c][input_c] = np.sum(self.Z[input_c][i:i+self.F[0],j:j+self.F[1]] * dA[output_c][i][j])

        # for文　input_c:元の入力、output_c:元の出力、j：Zの1次元入力サイズ、s:1次元フィルタサイズ

        for output_c in range(self.W.shape[0]):
            for input_c in range(self.W.shape[1]):
                for i in range(self.output_h):
                    for j in range(self.output_w):
                        self.dZ[input_c][i:self.F[0]+i, j:self.F[1]+j] += self.W[output_c][input_c] * dA[output_c][i][j]
                        
        self.dZ = self.dZ[:, 1:-1,1:-1]
        delta = self.dZ
        self = self.optimizer.update(self)
        
        return  delta


In [60]:
#cnn.add(SimpleConv2d(Conv_Initializer(), SGD2(0.01), F=[3, 3], S=[1,1], input_channels_num=1, output_channels_num=3))

clf = SimpleConv2d(Conv_Initializer(), SGD2(0.01), F=[3, 3], S=[1,1])
XX = np.ones([1,4,4])
clf.W = np.ones([3,1,3,3])
clf.B = np.ones([3,1,1])
temp11=clf.forward(XX)
temp11

array([[[10., 10.],
        [10., 10.]],

       [[10., 10.],
        [10., 10.]],

       [[10., 10.],
        [10., 10.]]])

In [61]:
clf.backward(temp11)

array([[[120., 120.,  60.,   0.],
        [120., 120.,  60.,   0.],
        [ 60.,  60.,  30.,   0.],
        [  0.,   0.,   0.,   0.]]])

## 【問題3】最大プーリング層の作成

In [62]:
class MaxPool2D:
    def __init__(self,F,S):
        self.F = (2,2)
        self.S = (2,2)

    def get_output_shape(self):
        self.output_h = int((self.Z.shape[1] - self.F[0]) / self.S[0] + 1)
        self.output_w = int((self.Z.shape[2] - self.F[1]) / self.S[1] + 1)
        return self.output_h, self.output_w
                    
    def forward(self, Z):
        self.Z = Z
        self.output_shape = self.get_output_shape()
        A = np.zeros([self.Z.shape[0], self.output_h, self.output_w])
        self.A_index = np.zeros([self.Z.shape[0], self.output_h, self.output_w, 2]).astype(int)
        for input_c in range(self.Z.shape[0]):
            for i in range(self.output_h):
                for j in range(self.output_w):
                    A[input_c][i,j]= np.max(self.Z[input_c][i*self.S[0]:i*self.S[0]+self.F[0], j*self.S[1]:j*self.S[1]+self.F[1]])
                    self.A_index[input_c][i,j] = np.unravel_index(np.argmax(self.Z[input_c][i*self.S[0]:i*self.S[0]+self.F[0],
                                                                                       j*self.S[1]:j*self.S[1]+self.F[1]]),(2,2))+np.array([i*self.S[0],j*self.S[1]])
        self.A = A
        assert(np.array(A).shape == np.zeros([self.Z.shape[0], self.output_h, self.output_w]).shape)
        return A
        
    def backward(self, dA):
        dZ = np.zeros([self.Z.shape[0],self.Z.shape[1],self.Z.shape[2]])
        for input_c in range(self.Z.shape[0]):
            for i in range(self.output_h):
                for j in range(self.output_w):
                    h = self.A_index[input_c][i,j][0]
                    w = self.A_index[input_c][i,j][1]
                    dZ[input_c][h,w]=self.A[input_c][i,j]
        
        return dZ
    


In [63]:
temp1 = np.array(range(16)).reshape(1,4,4)
mp = MaxPool2D((2,2),(2,2))
print(temp1)
mp.forward(temp1)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]]


array([[[ 5.,  7.],
        [13., 15.]]])

In [64]:
mp.backward(mp.forward(temp1))

array([[[ 0.,  0.,  0.,  0.],
        [ 0.,  5.,  0.,  7.],
        [ 0.,  0.,  0.,  0.],
        [ 0., 13.,  0., 15.]]])

## 【問題5】平滑化

In [65]:
class Flatten:

    def forward(self,Z):
        self.origin_shape = Z.shape
        self.Z = Z.reshape(1,-1)
        return self.Z
    def backward(self,dA):
        self.dA = dA.reshape(self.origin_shape)
        return self.dA
    

In [66]:
FL = Flatten()
print(FL.forward(temp1))
print(FL.backward(FL.forward(temp1)))

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]
  [12 13 14 15]]]


## 【問題6】学習と推定

##  <活性化関数、初期化、学習モード>

In [10]:
class Tanh:
    def __init__(self):
        self.out = None
    def forward(self,x):
#          out = 1/(1 + np.exp(-x))
        out = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1 - self.out**2) ### 確認。out?
#         dx = dout * (1.0 - self.out) * self.out
        return dx

In [8]:
class Softmax:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    def forward(self,x):
        self.y = np.exp(x)/ (np.sum(np.exp(x),axis=1).reshape(-1,1))
        return self.y
    def backward(self,t):
        self.t = t
        delta_sf = self.y -self.t
#       self.loss = self.cross_entropy_error(self.y, self.t)
        return delta_sf
    def cross_entropy_error(self,y,t):
        self.t = t
        c_entropy = -1/len(self.t)*(np.sum(self.t*np.log(self.y))) ### ｔは1hot_encoder
        return c_entropy

In [14]:
class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        layer.W = layer.W - self.lr * layer.dW
        layer.B = layer.B - self.lr * layer.dB.mean(axis = 0) 

In [16]:
class Xavier_Initializer:
    def __init__(self,xavier):
        self.xavier = None
    def WI(self, n_nodes1, n_nodes2):
        self.xavier = 1/math.sqrt(n_nodes1)
        WI = np.random.randn(n_nodes1, n_nodes2) * self.xavier
        return WI
    def BI(self, n_nodes2):
        BI = np.random.randn(n_nodes2) * self.xavier
        return BI

In [17]:
class SimpleInitializer:

    def __init__(self, sigma):
        self.sigma = sigma
    def WI(self, n_nodes1, n_nodes2):
        WI = np.random.randn(n_nodes1, n_nodes2) * self.sigma
        return WI
    def BI(self, n_nodes2):
        BI = np.random.randn(n_nodes2) * self.sigma
        return BI

In [18]:
class Xavier_Initializer2:
    def __init__(self,xavier):
        self.xavier = None
    def WI(self, n_samples, n_channels, hight, width):
        self.xavier = 1/math.sqrt(hight*width)
        WI = np.random.randn(n_samples, n_channels, hight, width) * self.xavier
        
        return WI
    def BI(self, n_nodes2):
        BI = np.random.randn(n_nodes2) * self.xavier
        return BI

## データセットの定義

In [71]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [72]:
### X_test y_testは使用しない？　###
X_train = X_train.astype(np.float).reshape(60000,28,28)
X_test = X_test.astype(np.float).reshape(10000,28,28)
X_train /= 255
X_test /= 255

In [73]:
X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

In [74]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

## 全結合層（flattenを利用）など

In [68]:
class FC:

    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        self.initializer = initializer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2        
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = self.initializer.WI(self.n_nodes1,self.n_nodes2)
        self.B = self.initializer.BI(self.n_nodes2)

    def forward(self, Z):
        self.flatten = Flatten()
        self.flat_Z = self.flatten.forward(Z)
        A = np.dot(self.flat_Z, self.W)+ self.B
        return A

    def backward(self, dA):
        self.dB = dA
        self.dW = np.dot(self.flat_Z.T, dA)
        dZ = self.flatten.backward(np.dot(dA, self.W.T))
        self = self.optimizer.update(self) 
        return dZ


In [69]:
class Scratch_Cnn2d_Classifier:        
    def __init__(self, verbose = True, iter_num=1, lr=0.002, sigma=0.02):
        self.verbose = verbose
        self.iter_num = iter_num
        self.lr = lr
        self.sigma = sigma
        self.network_layers = []
        
    def fit(self, X,y):
        
        for i in range(self.iter_num):
            for mini_X, mini_y in zip(X,y):
                mini_X =mini_X.reshape(1,28,28)
                self.forward_network(mini_X)
                self.back_network(mini_y)
                
    def add(self, l):
        self.network_layers.append(l)
        
    def forward_network(self,next_input):
        for l in self.network_layers:
            next_input = l.forward(next_input)
        return next_input
        
    def back_network(self,y):
        next_input = y
        for l in reversed(self.network_layers):
            next_input = l.backward(next_input)
            
    def predict(self,X):
      
        preds = []
        for x in X:
            pred = self.forward_network(x)
            preds.append(np.argmax(pred))
        return preds        

In [75]:
y_train.shape

(48000, 10)

In [95]:
cnn = Scratch_Cnn2d_Classifier()

cnn.add(SimpleConv2d(Conv_Initializer(), SGD2(0.01), F=[3, 3], S=[1,1], input_channels_num=1, output_channels_num=3))
cnn.add(MaxPool2D(F=(2, 2), S=2))
cnn.add(Tanh())
cnn.add(Tanh())
cnn.add(FC(507, 200, SimpleInitializer(0.02), SGD(0.02)))
cnn.add(Tanh())
cnn.add(FC(200, 10, SimpleInitializer(0.02), SGD(0.02)))
cnn.add(Softmax())

cnn.fit(X_train[0:2999],y_train[0:2999])

In [96]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test[0:300], cnn.predict(X_test[0:300]))


0.7766666666666666

## 【問題9】出力サイズとパラメータ数の計算


    入力サイズ : 144×144, 3チャンネル
    フィルタサイズ : 3×3, 6チャンネル
    ストライド : 1
    パディング : なし

パラメータ数　６×３×３×３＋６＝１６８  
出力サイズ　　１４２×１４２×６＝１２０９８４


    入力サイズ : 60×60, 24チャンネル
    フィルタサイズ : 3×3, 48チャンネル
    ストライド　: 1
    パディング : なし

パラメータ数　４８×２４×３×３＋４８＝１０４１６  
出力サイズ　　５８×５８×４８＝１６１４７２  


    入力サイズ : 20×20, 10チャンネル
    フィルタサイズ: 3×3, 20チャンネル
    ストライド : 2
    パディング : なし

パラメータ数　２０×１０×３×３＋２０＝１８２０  
出力サイズ　　９×９×２０＝１６２０  


※　以下はメモ

In [88]:
cnn.network_layers[0].dB

array([ 110.35090414,   17.42174294, -115.20198824])

In [92]:
cnn.network_layers[0].dB

array([-86.24966074,  74.85441542, -42.68688208])

In [97]:
cnn.network_layers[0].dB

array([-140.50912397,   15.4857399 ,  125.20330263])

In [87]:
cnn.network_layers[0].W

array([[[[-0.28160414, -0.15293035,  0.67173163],
         [-0.8886815 , -0.35494127, -0.99650568],
         [ 1.24010824, -0.45822004,  1.55409192]]],


       [[[ 0.56498781,  1.4503962 , -0.61505901],
         [-0.38846196, -1.19736636, -1.07165554],
         [-0.12624013, -0.15166334, -0.21587572]]],


       [[[-0.55059972,  0.281994  ,  0.88400459],
         [-1.02268013, -0.63442587, -0.70707607],
         [ 0.15991049,  0.89289107,  0.76495317]]]])

In [93]:
cnn.network_layers[0].W

array([[[[ 0.04022997, -2.46992965, -0.74870701],
         [ 1.33582507, -1.47829346, -1.48727196],
         [-1.5521729 , -0.35765659,  1.4609603 ]]],


       [[[ 0.64507137, -1.57538347, -0.11367834],
         [ 0.46592135, -0.42830212, -0.76913932],
         [ 0.82896859, -1.9493051 ,  0.22489649]]],


       [[[ 0.36050474,  0.15461494, -0.9170836 ],
         [ 0.26876445,  0.22766495,  0.19831994],
         [-0.14212663,  0.21903203,  1.1110323 ]]]])

In [98]:
cnn.network_layers[0].W

array([[[[-0.26118649,  0.91289749, -1.05585896],
         [-0.55666682,  0.16417285,  0.482402  ],
         [ 0.01573109, -1.38311144,  0.06238562]]],


       [[[ 0.13534149,  0.49818183,  0.51786218],
         [ 0.07937398,  0.19577898, -1.9428185 ],
         [-0.30047763, -0.7351651 , -0.48899004]]],


       [[[-0.86437243,  0.03029113,  1.32696095],
         [ 0.89828227, -0.17134098, -0.88884581],
         [-1.19681188,  0.30687879, -0.24680031]]]])

In [89]:
cnn.network_layers[0].B

array([[[ 0.41684991]],

       [[ 0.03256858]],

       [[-0.91995147]]])

In [94]:
cnn.network_layers[0].B

array([[[ 0.03885694]],

       [[ 0.77695952]],

       [[-0.33225684]]])

In [99]:
cnn.network_layers[0].B

array([[[-0.78547055]],

       [[ 0.09945419]],

       [[ 0.65748301]]])